<a href="https://colab.research.google.com/github/akshayjain777/CS584-Final-Project/blob/main/Image_Colorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/ImageColorization

/content/gdrive/MyDrive/ImageColorization


In [4]:
!ls

 code.py  'deep-koalarization-master 2'   images        README.md
 data	   download.py			  __pycache__


In [27]:
import os
import sys
from keras.layers import Conv2D, UpSampling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array,load_img
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
#import tensorflow.compat.v2 as tf
#print("tf version",tf.__version__)


folder_path = 'images/'

# #Normalize images - divide by 255
# train_datagen = ImageDataGenerator(rescale=1. / 255)


# #Resize images, if needed
# train = train_datagen.flow_from_directory(path, 
#                                         target_size=(256, 256), 
#                                         class_mode=None)


train = os.listdir(folder_path)

#Convert from RGB to Lab
"""
by iterating on each image, we convert the RGB to Lab. 
Think of LAB image as a grey image in L channel and all color info stored in A and B channels. 
The input to the network will be the L channel, so we assign L channel to X vector. 
And assign A and B to Y.
"""

X =[]
Y =[]
for img in train[0]:
  try:
      print("hello",img.shape)
      lab = rgb2lab(img)
      print("hello")
      X.append(lab[:,:,0]) 
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)


#Encoder

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

#Decoder
#Decoder
#Note: For the last layer we use tanh instead of Relu. 
#This is because we are colorizing the image in this layer using 2 filters, A and B.
#A and B values range between -1 and 1 so tanh (or hyperbolic tangent) is used
#as it also has the range between -1 and 1. 
#Other functions go from 0 to 1.
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])
model.summary()


model.fit(X,Y,validation_split=0.1, epochs=5, batch_size=16)

model.save('colorize_autoencoder.model')

error
error
error
error
error
error
error
(0, 1)
(0,)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_52 (Conv2D)          (None, 128, 128, 64)      640       
                                                                 
 conv2d_53 (Conv2D)          (None, 128, 128, 128)     73856     
                                                                 
 conv2d_54 (Conv2D)          (None, 64, 64, 128)       147584    
                                                                 
 conv2d_55 (Conv2D)          (None, 64, 64, 256)       295168    
                                                                 
 conv2d_56 (Conv2D)          (None, 32, 32, 256)       590080    
                                                                 
 conv2d_57 (Conv2D)          (None, 32, 32, 512)       1180160   
                                                                 


ValueError: ignored